In [33]:
import time
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

link = 'https://openlibrary.org/subjects'
r = requests.get(link)

soup = BeautifulSoup(r.text, 'html.parser')

### 1.Extracting Topic links

In [34]:
links = []

for i in tqdm(soup.find_all('li')):
  try:
    link = i.find('a').get('href')
    if (len(link.split('/')) == 3):
      link = 'https://openlibrary.org/search?subject=' + link.split('/')[-1].lower()

      link_ = link.replace('search?subject=','subjects/')
      r = requests.get(link_)
      soup = BeautifulSoup(r.text, 'html.parser')
      pages_ = int(soup.find('span', class_ = 'count').text.strip().split(' ')[0])//20

      links.append([link,pages_])
  except:
    pass

links = links[6:-5]

links = links[6:]

100%|██████████| 183/183 [02:19<00:00,  1.31it/s]


### 2. Scraping Full Data

In [ ]:
for link in links:
  url = link[0]
  pages = link[1]

  data = []

  for page in tqdm(range(1, pages + 1)):

    category = url.split('=')[-1]
    response = requests.get(url + '&page=' + str(page))
    # time.sleep(2)
    soup = BeautifulSoup(response.text, "html.parser")
    book_elements = soup.find_all("li", {"class": "searchResultItem"})

    for book in book_elements:
      try:
        try:
          title_element = book.find("h3")
        except:
          title_element = np.nan

        try:
          title = title_element.text.strip()
        except:
          title = np.nan

        try:
          title_id = title_element.find('a').get('href')
        except:
          title_id = np.nan

        try:
          author_element = book.find("span", class_ = 'bookauthor').find('a')
        except:
          author_element

        try:
          author = author_element.text.strip()
        except:
          author = np.nan

        try:
          author_id = author_element.get('href')
        except:
          author_id = np.nan

        try:
          publish_year = book.find('span', class_ = 'publishedYear').text.strip()
        except:
          publish_year = np.nan

        try:
          cover_url = book.find("img").get('src')
        except:
          cover_url = np.nan
      except:
        pass

      data.append([title, author, category, publish_year, title_id, author_id,cover_url])

  df = pd.DataFrame(data, columns = ['title', 'author','category', 'publish_year','title_id','author_id','cover_url'])

  df.to_csv(category + '.csv',index = False)

  4%|▍         | 156/3751 [06:24<2:08:09,  2.14s/it]